In [ ]:
from google.colab import files
files=files.upload()

Saving Train1.tsv to Train1.tsv
Saving Train2.tsv to Train2.tsv
Saving Train3.tsv to Train3.tsv
Saving Train3labels.txt to Train3labels.txt
Saving Ty.tsv to Ty.tsv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df1 = pd.read_csv('Train1.tsv', sep="\t", header=None ,names=["id","text","polarity"] )

In [ ]:
df1.shape

(14594, 3)

In [ ]:
df2=pd.read_csv('Train2.tsv', sep="\t", header=None ,names=["id","text","polarity"] )

In [ ]:
df2.shape

(3000, 3)

In [ ]:
df3=pd.read_csv('Train3.tsv', sep="\t", header=None ,names=["id","text"] )


In [ ]:
df3.shape

(3000, 2)

In [ ]:
ty = pd.read_csv('Train3labels.txt', sep=",", header=None )
sentiment = []
for i in range(len(ty)):
    if ty[1][i]=='negative':
        sentiment.append(0)
    elif ty[1][i]=='neutral':
        sentiment.append(1)
    elif ty[1][i]=='positive':
        sentiment.append(2)

ty = np.array(sentiment)

In [ ]:
df3['polarity']=ty

In [ ]:
df3['polarity'].value_counts()

1    1100
2    1000
0     900
Name: polarity, dtype: int64

In [ ]:
d=[df1,df2,df3]
df=pd.concat(d)

In [ ]:
df.shape

(20594, 3)

In [ ]:
df['polarity'].value_counts()

1    7701
2    6851
0    6042
Name: polarity, dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df.head()

,id,text,polarity
0,3,pakistan ka ghra tauq pakistan israel ko tasle...,0
1,41,madarchod mull ye mathura nahi dikha tha jab m...,0
2,48,manya pradhan mantri mahoday shriman narendra ...,2
3,64,krishna jcb full trend chal rahi aa nan,2
4,66,loksabha janta sirf modi ko vote de rahi thi n...,2


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf  = TfidfVectorizer(ngram_range=(1, 1), min_df=2)
features = tfidf.fit_transform(df['text'])

In [ ]:
type(features)

scipy.sparse.csr.csr_matrix

In [ ]:
sentiment=df['polarity']

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(features,sentiment,test_size=0.30, shuffle = False )

In [ ]:
X_train.shape

(14395, 16421)

In [ ]:
X_test.shape

(6170, 16421)

In [ ]:
y_test.shape

(6170,)

<h2>Support Vector Machines<h2>

In [ ]:
from sklearn import svm
lin_clf = svm.SVC(kernel='linear',decision_function_shape='ovr', class_weight='balanced',random_state=0)
lin_clf.fit(X_train, y_train)
y_pred = lin_clf.predict(X_test)
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
print("Accuracy Score = ", accuracy_score(y_test, y_pred))
print("F1-Score = ", f1_score(y_test, y_pred, average='macro'))
cm=confusion_matrix(y_test, y_pred)
print(cm)

Accuracy Score =  0.726742301458671
F1-Score =  0.7283280413578649
[[1523  347   94]
 [ 440 1409  376]
 [ 119  310 1552]]


In [ ]:
from sklearn.metrics import classification_report
print("SVM classification report")
print(classification_report(y_test, y_pred))

SVM classification report
              precision    recall  f1-score   support

           0       0.73      0.78      0.75      1964
           1       0.68      0.63      0.66      2225
           2       0.77      0.78      0.78      1981

    accuracy                           0.73      6170
   macro avg       0.73      0.73      0.73      6170
weighted avg       0.73      0.73      0.73      6170



<h2>Logistic Regression<h2>

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(random_state=0, class_weight='balanced')
log_model = log_model.fit(X_train, y_train)
y_pred = log_model.predict(X_test)
print("Accuracy Score = ", accuracy_score(y_test, y_pred))
print("F1-Score = ", f1_score(y_test, y_pred, average='macro'))
print(confusion_matrix(y_test, y_pred))

Accuracy Score =  0.7239870340356565
F1-Score =  0.7255206452279892
[[1516  343  105]
 [ 433 1405  387]
 [ 117  318 1546]]


In [ ]:
from sklearn.metrics import classification_report
print("Logistic regression classification report")
print(classification_report(y_test, y_pred))

Logistic regression classification report
              precision    recall  f1-score   support

           0       0.73      0.77      0.75      1964
           1       0.68      0.63      0.65      2225
           2       0.76      0.78      0.77      1981

    accuracy                           0.72      6170
   macro avg       0.72      0.73      0.73      6170
weighted avg       0.72      0.72      0.72      6170



<h2>Multinomial Naive bayes <h2>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train.toarray(), y_train)
y_pred = MNB.predict(X_test.toarray())
print("Accuracy Score = ", accuracy_score(y_test, y_pred))
print("F1-Score = ", f1_score(y_test, y_pred, average='macro'))
print(confusion_matrix(y_test, y_pred))

Accuracy Score =  0.7144246353322529
F1-Score =  0.7179642327216219
[[1377  495   92]
 [ 279 1525  421]
 [  93  382 1506]]


In [ ]:
from sklearn.metrics import classification_report
print("Multinomial bayes classification report")
print(classification_report(y_test, y_pred))

Multinomial bayes classification report
              precision    recall  f1-score   support

           0       0.79      0.70      0.74      1964
           1       0.63      0.69      0.66      2225
           2       0.75      0.76      0.75      1981

    accuracy                           0.71      6170
   macro avg       0.72      0.72      0.72      6170
weighted avg       0.72      0.71      0.72      6170



<h2>Randomforest classifier<h2>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10, random_state=0, class_weight='balanced')
import time
start_time=time.time()
model.fit(X_train, y_train) 
duration = time.time()-start_time
print(duration/60, " minutes")
y_pred = model.predict(X_test)
print("Accuracy Score = ", accuracy_score(y_test, y_pred))
print("F1-Score = ", f1_score(y_test, y_pred, average='macro'))
print(confusion_matrix(y_test, y_pred))

0.07796785036722818  minutes
Accuracy Score =  0.7418152350081038
F1-Score =  0.7450444468866424
[[1501  395   68]
 [ 396 1581  248]
 [ 115  371 1495]]


In [ ]:
from sklearn.metrics import classification_report
print("Random Forest Classifier classification report")
print(classification_report(y_test, y_pred))

Random Forest Classifier classification report
              precision    recall  f1-score   support

           0       0.75      0.76      0.76      1964
           1       0.67      0.71      0.69      2225
           2       0.83      0.75      0.79      1981

    accuracy                           0.74      6170
   macro avg       0.75      0.74      0.75      6170
weighted avg       0.75      0.74      0.74      6170



<h2> Ensemble model <h2>

In [ ]:
    start_time=time.time()
    estimator = [] 
    estimator.append(('SVC',  
                      svm.SVC(kernel='linear',decision_function_shape='ovr', class_weight='balanced',random_state=0, probability=True))) 
    estimator.append(('LR', LogisticRegression(random_state=0, class_weight='balanced'))) 
    estimator.append(('MNB', MultinomialNB()))
    estimator.append(('RF', RandomForestClassifier(n_estimators=250, random_state=0, class_weight='balanced')))

    vot_soft = VotingClassifier(estimators = estimator, voting ='soft',) 
    vot_soft.fit(X_train, y_train) 
    y_pred = vot_soft.predict(X_test) 


    duration = time.time()-start_time
    print(duration/60, " minutes")

    print("Accuracy Score = ", accuracy_score(y_test, y_pred))
    print("F1-Score = ", f1_score(y_test, y_pred, average='macro'))
    print(confusion_matrix(y_test, y_pred))
    print("Random Forest Classifier classification report")
    print(classification_report(y_test, y_pred))


4.749778254826864  minutes
Accuracy Score =  0.7687196110210697
F1-Score =  0.7713175036015744
[[1505  378   81]
 [ 287 1635  303]
 [  71  307 1603]]
Random Forest Classifier classification report
              precision    recall  f1-score   support

           0       0.81      0.77      0.79      1964
           1       0.70      0.73      0.72      2225
           2       0.81      0.81      0.81      1981

    accuracy                           0.77      6170
   macro avg       0.77      0.77      0.77      6170
weighted avg       0.77      0.77      0.77      6170

